In [24]:
%%time
import numpy as np
import math
import cmath as cv


flag=1 # живой доф
death_time=10 #мертвое время в периодах
p_dark_count=0.0000003
ef_det=0.1 # эффективность детектора в случае прилета 1 фотона
t=0.4 #коэффициент пропускания
p_opt=0.01 # неточность в приготовлении состояний 
mu=0.3 #матожидание фотонов в импульсе
lenth=28000 # длина последовательности
s=0 #счетчик мертвого времени


def bit2pulse_z_basis(bit_value, power, phase): #превращаем бит в импульс
    if bit_value == 0:
        return [cv.rect(math.sqrt(power), phase), cv.rect(math.sqrt(0.012), phase)] #если 0 то early
    elif bit_value == 1:
        return [cv.rect(math.sqrt(0.012), phase), cv.rect(power, phase)] #если 1 то late
    else:
        print("Incorrect input")
        return [0]
    
    
def detector_probability(pulse):
    power = abs(pulse)**2 # подсчитываем мощность импульса
    t=1 #коэффициент пропускания
    ef_det=0.1 # эффективность детектора в случае прилета 1 фотона
    probability = 1 -math.exp(-power*t*ef_det) # вероятность детектирования импульса с мощностью power
    return probability

    
    
def post_processing_z_Bob(Charlie_data):
    summ = 0
    for i in range(len(Charlie_data)): #смотрим сколько детекторов сработало
        summ += Charlie_data[i]
    if summ != 2: #если сработало не 2 детектора, то нам не интересно
        return -1
    else:
        if len(Charlie_data) < 4: #проверка на дурака
            print("Incorrect Charlie data")
            return -1
        elif (Charlie_data[0] == Charlie_data[2]): #сработали оба early
            return -1
        elif (Charlie_data[1] == Charlie_data[3]): #сработали оба late
            return -1
        else: # сработали разные детекторы
            return 1
    
    
def post_processing_z_single_click(Charlie_data, Alice_bit, Bob_bit):
    summ = 0
    print(Charlie_data)
    for i in range(len(Charlie_data)): #смотрим сколько детекторов сработало
        summ += Charlie_data[i]
    if summ >= 2 or summ < 0: #если сработало не 2 детектора, то нам не интересно (дописал >=, почему не интересно?)
        return -1
    if (Alice_bit != Bob_bit):
        return -1
    else:
        if (Charlie_data[0] == 1 or Charlie_data[2] == 1) and Alice_bit == 1:  #поменял на ==
            return 2
        elif (Charlie_data[1] == 1 or Charlie_data[3] == 1) and Alice_bit == 0:
            return 2
        else:
            return 1
        
        
        
            
def Charlie(pulse_a, pulse_b, spd_status):
    ready_to_work = 0 # флаг того, что все дофы готовы к работе
    for spd in range(len(spd_status)): #проверяем когда восстановятся все дофы
        if spd_status[spd] < ready_to_work:
            ready_to_work = spd_status[spd]
    if (ready_to_work < 0): #пропускаем цикл на восстановление дофа
        for spd in range(len(spd_status)):
            if spd_status[spd] < 0:
                spd_status[spd] += 1    
        
    for spd in range(len(spd_status)): #смотрим на темновое срабатывание
        if (spd_status[spd] > -1):
            spd_status[spd] = np.random.choice([0, -death_time], p=[1-p_dark_count, p_dark_count])
    
    pulse_c, pulse_d = beam_splitter(pulse_a,  pulse_b) #разделение на первом BS
    pulse_e1, pulse_l1 = beam_splitter(pulse_c, [0.0, 0.0]) #Разделение на BS первого плеча
    pulse_e2, pulse_l2 = beam_splitter(pulse_d, [0.0, 0.0]) #Разделение на BS второго плеча
    all_pulses = [] #распределение импульсов на детекторы
    all_pulses.append(pulse_e1)
    all_pulses.append(pulse_l1)
    all_pulses.append(pulse_e2)
    all_pulses.append(pulse_l2)
    
    if (ready_to_work == 0): #если все готовы к работе, то смотрим на возможные срабатывания детекторов
        for i in range(len(spd_status)):
            if spd_status[i] == 0:
                  spd_status[i] = np.random.choice([0, 1], p=[1-detector_probability(all_pulses[i][i % 2]), detector_probability(all_pulses[i][i % 2])])

    
    return spd_status
        
    

#def p_detection_count(mu = 0.3, max_count = 5, ef_det = 0.1, t = 0.4): #Подсчитывает эффективность детектирования импульсов
    # mu - матожидание фотонов в импульсе, max_count - максимум фотонов в импульсе
#   p_detection = 0
#   for m in range(1, max_count):
#        p_detection += (math.exp(-mu * t) * ((mu * t)**m) / math.factorial(m)) * (1 - (1 - ef_det)**m)
#    return p_detection


def QBER_count(Alice, Bob): #Подсчет кубера из сырого ключа Алисы и Боба
    #Alice - сырой ключ Алисы, Bob - сырой ключ Боба
    if len(Alice) != len(Bob):
        print("Incorrect input")
        return 1
    else:
        N = 0
        Error = 0
        for i in range(len(Alice)):
            if ((Bob[i] == 1)or(Bob[i] == 0)):
                N += 1
                if (Alice[i] != Bob[i]):
                    Error += 1
        if N != 0:
            QBER = Error/N
            print("QBER = ", QBER, "N = ", N, "Error = ", Error)
            return QBER
        else:
            print("Not enough data, try with more length")
            return 1
        
def QBER_single_count(data): #Подсчет кубера из сырого ключа Алисы и Боба
    #Alice - сырой ключ Алисы, Bob - сырой ключ Боба
    N = 0
    Error = 0
    for i in range(len(data)):
        if ((data[i] == 1)or(data[i] == 2)):
            N += 1
            if (data[i] == 2):
                Error += 1
    if N != 0:
        QBER = Error/N
        print("QBER = ", QBER, "N = ", N, "Error = ", Error)
        return QBER
    else:
        print("Not enough data, try with more length")
        return 1

    
#print(post_processing_z_single_click([0,0,1,0], 1, 1))




        
def beam_splitter(pulse_a, pulse_b): #импульсы на выходе BS
    pulse_c = [(pulse_a[0] + pulse_b[0]) / math.sqrt(2), (pulse_a[1] + pulse_b[1]) / math.sqrt(2)]
    pulse_d = [(pulse_a[0] - pulse_b[0]) / math.sqrt(2), (pulse_a[1] - pulse_b[1]) / math.sqrt(2)]

    return pulse_c, pulse_d

#pulse_wq, pulse_qw = beam_splitter([cv.rect(0.9, 0.0), 0], [0, cv.rect(0.9, 1.5)])

#print(pulse_wq)
#p_detection = p_detection_count(mu, 5, 0.1, t) считаем p_detection
#print(p_detection) 
#print(detector_probability(1000, p_detection))

#print(post_processing_z_Bob([0, 1, 0, 1]))

#print(Charlie(pulse_qw, pulse_wq, [0, 0, 0, 0], p_detection))
#print(post_processing_z_Bob(Charlie(pulse_qw, pulse_wq, [0, 0, 0, 0], p_detection)))

        
        
        
Alice_bit_values = np.random.randint(0, 2, lenth) #создаем массив битов Алисы
Bob_bit_values = np.random.randint(0, 2, lenth) #создаем массив битов Боба




    
spd_status = [0, 0, 0, 0] #состояние дофов
alice_final_key = []  #сырой ключ Алисы
bob_final_key = [] #сырой ключ Боба
data_for_single_QBER = []
spd_clicks = [0, 0, 0, 0]
    
for k in range(lenth):
    phase_a = 2*math.pi*np.random.random() #рандомизированная фаза импульса
    pulse_a = bit2pulse_z_basis(Alice_bit_values[k], mu*t, phase_a) #создаем импульс от Алисы
    #print(pulse_a)
    phase_b = 2*math.pi*np.random.random()
    pulse_b = bit2pulse_z_basis(Bob_bit_values[k], mu*t, phase_b) #создаем испульс от Боба
    #print(pulse_b)
    

    for spd in range(len(spd_status)): # если доф сработал, то он начинает отдыхать
        if (spd_status[spd] == 1):
            #print("hi")
            spd_status[spd] -= death_time

    
    spd_status = Charlie(pulse_a, pulse_b, spd_status) #одна итерация
    
    #print(spd_status)
    for i in range(len(spd_status)):
        if spd_status[i] == 1:
            spd_clicks[i] += 1
    
    if(post_processing_z_Bob(spd_status) == 1): #подсчитываем полученный ключ и записываем его в сырые ключи
        alice_final_key.append(Alice_bit_values[k])
        bob_final_key.append(Bob_bit_values[k]^1)
        #print(Alice_bit_values[k])
        #print(Bob_bit_values[k]^1)
    data_for_single_QBER.append(post_processing_z_single_click(spd_status, Alice_bit_values[k], Bob_bit_values[k]))
    
print(spd_clicks)
    
print(alice_final_key)
print(bob_final_key)

    
    

QBER_count(alice_final_key, bob_final_key) # подсчитываем QBER и выводим его
QBER_single_count(data_for_single_QBER)
        
    















[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 1, 0]
[0, 0, -8, 0]
[0, 0, -7, 0]
[0, 0, -6, 0]
[0, 0, -5, 0]
[0, 0, -4, 0]
[0, 0, -3, 0]
[0, 0, -2, 0]
[0, 0, -1, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 

0.0014366729678638942